# Imports

In [ ]:
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft, ifft
from scipy.stats import pearsonr
from scipy import signal as sig
from scipy.signal import ellip,lfilter,filtfilt,find_peaks,firwin,kaiserord, decimate, periodogram
from copy import deepcopy
from scipy import stats
import scipy
import warnings
warnings.filterwarnings('ignore')
from scipy.fft import rfft, rfftfreq
from scipy import signal
from scipy.integrate import simps
from sklearn. decomposition import PCA
from scipy.interpolate import CubicSpline, splrep, splev
from sklearn.linear_model import LinearRegression
from scipy.io import savemat
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import scale 
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler #standard scaler worked best
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers.core import Dense
from tensorflow.keras.layers import ReLU, LeakyReLU, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Conv1D
import joblib
from google.colab import files

# Loading data

In [ ]:
fs = 1000 #Hz

train_data = loadmat('/content/raw_training_data.mat')
train_dg = train_data['train_dg']
train_ecog = train_data['train_ecog']

test_data = loadmat('leaderboard_data.mat')['leaderboard_ecog']

# Removing bad channels

In [ ]:
#subject 1
#bad channels are : 54
train_ecog[0][0] = np.delete(train_ecog[0][0], 54, 1)
test_data[0][0] = np.delete(test_data[0][0], 54, 1)

#subject 2
#bad channels are : 20 and 37
train_ecog[1][0] = np.delete(train_ecog[1][0], [20,37], 1)
test_data[1][0] = np.delete(test_data[1][0], [20,37], 1)

#subject 3
# bad channels are : None

# Train test split

In [ ]:
#TRAIN SET
#train labels
train_dg1 = train_dg[0][0]
train_dg2 = train_dg[1][0]
train_dg3 = train_dg[2][0]

#train data
train_ecog1 = train_ecog[0][0]
train_ecog2 = train_ecog[1][0]
train_ecog3 = train_ecog[2][0]

#TEST SET
#test data
test_ecog1 = test_data[0][0]
test_ecog2 = test_data[1][0]
test_ecog3 = test_data[2][0]

# Preprocessing (filter, feature calc, windowing and R matrix function calc)

In [ ]:
def filter_data_2(raw_eeg, fs=1000):
  nyq=fs/2
  ripple_db=60.0 #ripple size for cutoff
  width=5.0/nyq #transition wid
  N, beta = kaiserord(ripple_db, width) #kaiser parameters
  taps_default=firwin(N,[4, 200],window=('kaiser',beta),pass_zero='bandpass',fs=1000)
  filtered_2=filtfilt(taps_default,1.0,raw_eeg)
  return filtered_2

def filternotch(x,fs=1000):
  b1, a1 = sig.iirnotch(60, 20, fs)
  b2, a2 = sig.iirnotch(120, 20, fs)
  b3, a3 = sig.iirnotch(180, 20, fs)
  output1=sig.filtfilt(b1,a1,x)
  output2=sig.filtfilt(b2,a2,output1)
  output3=sig.filtfilt(b3,a3,output2)
  return output3

In [ ]:
def NumWins(x,fs,winLen,winDisp):
  M =np.floor(((len(x)/fs) - winLen + winDisp)/ winDisp)
  return int(M)

def NumWins_no_overlap(x,winLen):
  M =np.divide((len(x)/fs), winLen)
  return int(M)

#feature 1
def Avg_voltage(x):
  avg_vol = np.mean(x)
  return avg_vol

#feature 2
def LL(x):
  ll_x = np.sum(np.absolute(np.ediff1d(x)))
  return ll_x

#feature 3
def Energy(x):
  E_x = np.sum(np.square(x))
  return E_x

#feature 4 ,5, 6, 7, 8
def Avg_Freq(x, fi, ff):
  #Convert to frequency domain:
  freq_sig = rfft(x)
  N = len(x)
  tdomain=rfftfreq(N,1/fs)  
  indices = np.where((tdomain>=fi) & (tdomain<=ff))[0]
  return np.mean(np.abs(freq_sig[indices]))

In [ ]:
def get_features(filtered_window, fs=1000):

  channels = np.shape(filtered_window)[1]
  features = np.empty([channels, 8])
  for ch in range(channels):
    feat1 = Avg_voltage(filtered_window[:,ch])
    feat2 = LL(filtered_window[:,ch])
    feat3 = Energy(filtered_window[:,ch])
    feat4 = Avg_Freq(filtered_window[:,ch], 5, 15)
    feat5 = Avg_Freq(filtered_window[:,ch], 20, 25)
    feat6 = Avg_Freq(filtered_window[:,ch], 75, 115)
    feat7 = Avg_Freq(filtered_window[:,ch], 125, 160)
    feat8 = Avg_Freq(filtered_window[:,ch], 160, 175)

    features[ch,:] = [feat1, feat2, feat3, feat4, feat5, feat6, feat7, feat8]
   
  features = np.reshape(features,(channels*8))

  return features

In [ ]:
def get_windowed_feats(raw_ecog, fs, window_length, window_overlap):

  filtered_eeg = np.empty(np.shape(raw_ecog))
  
  for ch in range(np.shape(raw_ecog)[1]):
    filtered_eeg[:,ch] = filternotch(filter_data_2(raw_ecog[:,ch]))  

  M = NumWins(filtered_eeg, fs, window_length, window_overlap)
  xLen = len(filtered_eeg)
  
  L = window_length
  d = window_overlap

  feature_vector = []
  for i in range(int(M)):
    feature_values = get_features(filtered_eeg[int(xLen - ((L+i*d)*fs)) : int(xLen-(i*d*fs)), :])
    feature_vector.append(feature_values)
  feature_vector = np.array(feature_vector)
  feature_vector=feature_vector[::-1,:]

  return feature_vector

In [ ]:
def get_windowed_dg(raw_dg, fs, window_length, window_overlap):

  M = NumWins(raw_dg, fs, window_length, window_overlap)
  xLen = len(raw_dg)
  L = window_length
  d = window_overlap

  downsampled_dg = np.empty((M, 5))
  for i in range(int(M)):
    for ch in range(5):
      #feat = np.mean(raw_dg[int(xLen - ((L+i*d)*fs)) : int(xLen-(i*d*fs)), ch])             #option1
      feat = np.max(raw_dg[int(xLen - ((L+i*d)*fs)) : int(xLen-(i*d*fs)), ch])               #option2 - BEST FINAL
      #feat = signal.decimate(raw_dg[int(xLen - ((L+i*d)*fs)) : int(xLen-(i*d*fs)), ch], M)  #option3
      downsampled_dg[i,ch] = feat

  downsampled_dg=downsampled_dg[::-1,:]
  return downsampled_dg

In [ ]:
def create_R_matrix(features, N_wind):

  M, ch = np.shape(features)
  feats2 = np.empty([M+N_wind-1, ch])
  
  feats2[N_wind-1:,:] = deepcopy(features)
  for i in range(0,N_wind-1):
    feats2[i, :] = deepcopy(features[i,:])
  
  R = np.empty([M, ch*N_wind + 1])
  R[:,0] = np.ones((M))
  
  for i in range(N_wind-1, np.shape(feats2)[0]):
    temp_arr = []
    for n in np.arange(0,N_wind)[::-1]:
      temp_arr = np.concatenate((temp_arr, feats2[i-(n),:]), axis=None)

    R[i-(N_wind-1), 1:] = temp_arr

  return R

# X_train, X_test, y_train and R_train, R_test for each subject

In [ ]:
#----------SUBJECT 1-------------

#X_train
X_train_1 = get_windowed_feats(train_ecog1, 1000, 0.1,0.05)
X_train_1 = np.nan_to_num(X_train_1)

#X_test
X_test_1 = get_windowed_feats(test_ecog1, 1000, 0.1,0.05)
X_test_1 = np.nan_to_num(X_test_1)

#y_train
y_train_1 = get_windowed_dg(train_dg1, 25, 6, 2)

#R calc
R1_train = create_R_matrix(X_train_1, 3)
R1_test = create_R_matrix(X_test_1, 3)

In [ ]:
#----------SUBJECT 2-------------

#X_train
X_train_2 = get_windowed_feats(train_ecog2, 1000, 0.1,0.05)
X_train_2 = np.nan_to_num(X_train_2)

#X_test
X_test_2 = get_windowed_feats(test_ecog2, 1000, 0.1,0.05)
X_test_2 = np.nan_to_num(X_test_2)

#y_train
y_train_2 = get_windowed_dg(train_dg2, 25, 6, 2)

#R calc
R2_train = create_R_matrix(X_train_2, 3)
R2_test = create_R_matrix(X_test_2, 3)

In [ ]:
#----------SUBJECT 3-------------

#X_train
X_train_3 = get_windowed_feats(train_ecog3, 1000, 0.1,0.05)
X_train_3 = np.nan_to_num(X_train_3)

#X_test
X_test_3 = get_windowed_feats(test_ecog3, 1000, 0.1,0.05)
X_test_3 = np.nan_to_num(X_test_3)

#y_train
y_train_3 = get_windowed_dg(train_dg3, 25, 6, 2)

#R calc
R3_train = create_R_matrix(X_train_3, 3)
R3_test = create_R_matrix(X_test_3, 3)

# Neural network method 

## subject 1

In [ ]:
#-----------Subject 1-------------

#scaling (normalizing) the data
scaler1 = StandardScaler()
R1_train = scaler1.fit_transform(R1_train)

scaler_filename = "scaler1.save"
joblib.dump(scaler1, scaler_filename) 

R1_test = scaler1.transform(R1_test) 

In [ ]:
#creating the neural network architecture
model = Sequential()
model.add(Dense(128, input_dim = R1_train.shape[1], activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(1, activation = 'linear'))

#choosing optimizer and performance metric
model.compile(optimizer = 'adam', loss = 'mean_absolute_error')

#early stopping
early_stop = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose=1, patience = 25)

In [ ]:
#finger 1

#model.fit with early stopping
model.fit(R1_train, y_train_1[:,0], epochs = 500, batch_size = 32, verbose = 1, validation_split = 0.1, callbacks = [early_stop])
model.save('model11.h5')
files.download('model11.h5')
#predictions
test_pred11 = model.predict(R1_test)

Epoch 1/500
169/169 [==============================] - 3s 3ms/step - loss: 0.5813 - val_loss: 1.4846
Epoch 2/500
169/169 [==============================] - 0s 3ms/step - loss: 0.5259 - val_loss: 1.3489
Epoch 3/500
169/169 [==============================] - 0s 3ms/step - loss: 0.4477 - val_loss: 1.3465
Epoch 4/500
169/169 [==============================] - 0s 3ms/step - loss: 0.3850 - val_loss: 1.3295
Epoch 5/500
169/169 [==============================] - 0s 3ms/step - loss: 0.3274 - val_loss: 1.3791
Epoch 6/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2931 - val_loss: 1.2133
Epoch 7/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2680 - val_loss: 1.2685
Epoch 8/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2341 - val_loss: 1.3316
Epoch 9/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2190 - val_loss: 1.3111
Epoch 10/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1976 - val_lo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#finger 2

#model.fit with early stopping
model.fit(R1_train, y_train_1[:,1], epochs = 500, batch_size = 32, verbose = 1, validation_split = 0.1, callbacks = [early_stop])
model.save('model12.h5')
files.download('model12.h5')
#predictions
test_pred12 = model.predict(R1_test)

Epoch 1/500
169/169 [==============================] - 0s 3ms/step - loss: 0.6783 - val_loss: 0.6394
Epoch 2/500
169/169 [==============================] - 0s 3ms/step - loss: 0.5543 - val_loss: 0.5672
Epoch 3/500
169/169 [==============================] - 0s 3ms/step - loss: 0.4437 - val_loss: 0.5942
Epoch 4/500
169/169 [==============================] - 0s 3ms/step - loss: 0.3721 - val_loss: 0.5777
Epoch 5/500
169/169 [==============================] - 0s 3ms/step - loss: 0.3110 - val_loss: 0.5760
Epoch 6/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2620 - val_loss: 0.5614
Epoch 7/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2335 - val_loss: 0.5587
Epoch 8/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2002 - val_loss: 0.5812
Epoch 9/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1857 - val_loss: 0.5663
Epoch 10/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1709 - val_lo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#finger 3

#model.fit with early stopping
model.fit(R1_train, y_train_1[:,2], epochs = 500, batch_size = 64, verbose = 1, validation_split = 0.1, callbacks = [early_stop])
model.save('model13.h5')
files.download('model13.h5')
#predictions
test_pred13 = model.predict(R1_test)

Epoch 1/500
85/85 [==============================] - 0s 3ms/step - loss: 0.6870 - val_loss: 0.1600
Epoch 2/500
85/85 [==============================] - 0s 3ms/step - loss: 0.5777 - val_loss: 0.1641
Epoch 3/500
85/85 [==============================] - 0s 3ms/step - loss: 0.5490 - val_loss: 0.1773
Epoch 4/500
85/85 [==============================] - 0s 3ms/step - loss: 0.5175 - val_loss: 0.1705
Epoch 5/500
85/85 [==============================] - 0s 3ms/step - loss: 0.4798 - val_loss: 0.1773
Epoch 6/500
85/85 [==============================] - 0s 3ms/step - loss: 0.4358 - val_loss: 0.1952
Epoch 7/500
85/85 [==============================] - 0s 3ms/step - loss: 0.4027 - val_loss: 0.1969
Epoch 8/500
85/85 [==============================] - 0s 3ms/step - loss: 0.3544 - val_loss: 0.2576
Epoch 9/500
85/85 [==============================] - 0s 3ms/step - loss: 0.3191 - val_loss: 0.2433
Epoch 10/500
85/85 [==============================] - 0s 3ms/step - loss: 0.2796 - val_loss: 0.2873
Epoch 11/

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#finger 4

#model.fit with early stopping
model.fit(R1_train, y_train_1[:,3], epochs = 500, batch_size = 32, verbose = 1, validation_split = 0.1, callbacks = [early_stop])
model.save('model14.h5')
files.download('model14.h5')
#predictions
test_pred14 = model.predict(R1_test)

Epoch 1/500
169/169 [==============================] - 0s 3ms/step - loss: 0.6145 - val_loss: 0.6703
Epoch 2/500
169/169 [==============================] - 0s 3ms/step - loss: 0.4641 - val_loss: 0.7082
Epoch 3/500
169/169 [==============================] - 0s 3ms/step - loss: 0.3766 - val_loss: 0.6258
Epoch 4/500
169/169 [==============================] - 0s 3ms/step - loss: 0.3115 - val_loss: 0.6717
Epoch 5/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2676 - val_loss: 0.6823
Epoch 6/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2313 - val_loss: 0.6686
Epoch 7/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2082 - val_loss: 0.6566
Epoch 8/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1844 - val_loss: 0.6451
Epoch 9/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1718 - val_loss: 0.6581
Epoch 10/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1566 - val_lo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#finger 5

#model.fit with early stopping
model.fit(R1_train, y_train_1[:,4], epochs = 500, batch_size = 64, verbose = 1, validation_split = 0.1, callbacks = [early_stop])
model.save('model15.h5')
files.download('model15.h5')
#predictions
test_pred15 = model.predict(R1_test)

Epoch 1/500
85/85 [==============================] - 0s 3ms/step - loss: 0.6884 - val_loss: 0.2540
Epoch 2/500
85/85 [==============================] - 0s 3ms/step - loss: 0.4171 - val_loss: 0.2577
Epoch 3/500
85/85 [==============================] - 0s 3ms/step - loss: 0.3316 - val_loss: 0.2952
Epoch 4/500
85/85 [==============================] - 0s 3ms/step - loss: 0.2693 - val_loss: 0.2593
Epoch 5/500
85/85 [==============================] - 0s 3ms/step - loss: 0.2319 - val_loss: 0.3025
Epoch 6/500
85/85 [==============================] - 0s 3ms/step - loss: 0.2142 - val_loss: 0.3353
Epoch 7/500
85/85 [==============================] - 0s 3ms/step - loss: 0.1869 - val_loss: 0.2914
Epoch 8/500
85/85 [==============================] - 0s 3ms/step - loss: 0.1693 - val_loss: 0.3421
Epoch 9/500
85/85 [==============================] - 0s 3ms/step - loss: 0.1602 - val_loss: 0.3744
Epoch 10/500
85/85 [==============================] - 0s 3ms/step - loss: 0.1435 - val_loss: 0.3240
Epoch 11/

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#test_pred11 = test_pred11.reshape((-1,))
#test_pred12 = test_pred12.reshape((-1,))
#test_pred13 = test_pred13.reshape((-1,))
#test_pred14 = test_pred14.reshape((-1,))
#test_pred15 = test_pred15.reshape((-1,))

test_pred1 = np.vstack((test_pred11, test_pred12, test_pred13, test_pred14, test_pred15)).T

xs = np.linspace(0,test_pred1.shape[0],test_ecog1.shape[0])

#interpolation
y1 = np.empty_like(test_pred1[:,0]) 
cs1 = [] 
for i in range(test_pred1.shape[1]):
  x1 = np.arange(test_pred1.shape[0])
  y1 = test_pred1[:,i]
  cs1.append(CubicSpline(x1, y1, bc_type = 'clamped'))

interp_pred1_nn = np.vstack((cs1[0](xs),cs1[1](xs), cs1[2](xs), cs1[3](xs), cs1[4](xs))).T

## subject 2

In [ ]:
#-----------Subject 2-------------

#scaling (normalizing) the data
scaler2 = StandardScaler()
R2_train = scaler2.fit_transform(R2_train)

scaler_filename = "scaler2.save"
joblib.dump(scaler2, scaler_filename) 

R2_test = scaler2.transform(R2_test) 

In [ ]:
#creating the neural network architecture
model = Sequential()
model.add(Dense(128, input_dim = R2_train.shape[1], activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(1, activation = 'linear'))

#choosing optimizer and loss
model.compile(optimizer = 'adam', loss = 'mean_absolute_error')

#early stopping
early_stop = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose=1, patience = 25)

In [ ]:
#finger 1

#model.fit with early stopping
model.fit(R2_train, y_train_2[:,0], epochs = 500, batch_size = 32, verbose = 1, validation_split = 0.1, callbacks = [early_stop])
model.save('model21.h5')
files.download('model21.h5')
#predictions
test_pred21 = model.predict(R2_test)

Epoch 1/500
169/169 [==============================] - 1s 3ms/step - loss: 0.5021 - val_loss: 0.4248
Epoch 2/500
169/169 [==============================] - 0s 3ms/step - loss: 0.4353 - val_loss: 0.3962
Epoch 3/500
169/169 [==============================] - 0s 3ms/step - loss: 0.3763 - val_loss: 0.4179
Epoch 4/500
169/169 [==============================] - 0s 3ms/step - loss: 0.3296 - val_loss: 0.4289
Epoch 5/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2771 - val_loss: 0.4356
Epoch 6/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2518 - val_loss: 0.4303
Epoch 7/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2216 - val_loss: 0.4335
Epoch 8/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2051 - val_loss: 0.4481
Epoch 9/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1817 - val_loss: 0.4481
Epoch 10/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1728 - val_lo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#finger 2

#model.fit with early stopping
model.fit(R2_train, y_train_2[:,1], epochs = 500, batch_size = 32, verbose = 1, validation_split = 0.1, callbacks = [early_stop])
model.save('model22.h5')
files.download('model22.h5')
#predictions
test_pred22 = model.predict(R2_test)

Epoch 1/500
169/169 [==============================] - 0s 3ms/step - loss: 0.5754 - val_loss: 0.8235
Epoch 2/500
169/169 [==============================] - 0s 3ms/step - loss: 0.4938 - val_loss: 0.8390
Epoch 3/500
169/169 [==============================] - 0s 3ms/step - loss: 0.4380 - val_loss: 0.7907
Epoch 4/500
169/169 [==============================] - 0s 3ms/step - loss: 0.3841 - val_loss: 0.8103
Epoch 5/500
169/169 [==============================] - 0s 3ms/step - loss: 0.3319 - val_loss: 0.9260
Epoch 6/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2951 - val_loss: 0.8688
Epoch 7/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2397 - val_loss: 0.8693
Epoch 8/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2193 - val_loss: 0.8769
Epoch 9/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1974 - val_loss: 0.8695
Epoch 10/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1744 - val_lo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#finger 3

#model.fit with early stopping
model.fit(R2_train, y_train_2[:,2], epochs = 500, batch_size = 64, verbose = 1, validation_split = 0.1, callbacks = [early_stop])
model.save('model23.h5')
files.download('model23.h5')
#predictions
test_pred23 = model.predict(R2_test)

Epoch 1/500
85/85 [==============================] - 0s 3ms/step - loss: 0.6581 - val_loss: 0.2994
Epoch 2/500
85/85 [==============================] - 0s 3ms/step - loss: 0.5703 - val_loss: 0.3363
Epoch 3/500
85/85 [==============================] - 0s 3ms/step - loss: 0.5386 - val_loss: 0.3967
Epoch 4/500
85/85 [==============================] - 0s 3ms/step - loss: 0.4928 - val_loss: 0.5420
Epoch 5/500
85/85 [==============================] - 0s 3ms/step - loss: 0.4370 - val_loss: 0.5708
Epoch 6/500
85/85 [==============================] - 0s 3ms/step - loss: 0.3636 - val_loss: 0.5257
Epoch 7/500
85/85 [==============================] - 0s 3ms/step - loss: 0.3035 - val_loss: 0.6046
Epoch 8/500
85/85 [==============================] - 0s 3ms/step - loss: 0.2604 - val_loss: 0.5014
Epoch 9/500
85/85 [==============================] - 0s 3ms/step - loss: 0.2328 - val_loss: 0.5299
Epoch 10/500
85/85 [==============================] - 0s 3ms/step - loss: 0.1982 - val_loss: 0.6672
Epoch 11/

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#finger 4

#model.fit with early stopping
model.fit(R2_train, y_train_2[:,3], epochs = 500, batch_size = 32, verbose = 1, validation_split = 0.1, callbacks = [early_stop])
model.save('model24.h5')
files.download('model24.h5')
#predictions
test_pred24 = model.predict(R2_test)

Epoch 1/500
169/169 [==============================] - 0s 3ms/step - loss: 0.4697 - val_loss: 0.7682
Epoch 2/500
169/169 [==============================] - 0s 3ms/step - loss: 0.3806 - val_loss: 0.7296
Epoch 3/500
169/169 [==============================] - 0s 3ms/step - loss: 0.3179 - val_loss: 0.7010
Epoch 4/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2759 - val_loss: 0.6894
Epoch 5/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2416 - val_loss: 0.7088
Epoch 6/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2171 - val_loss: 0.6932
Epoch 7/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1937 - val_loss: 0.6859
Epoch 8/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1700 - val_loss: 0.6854
Epoch 9/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1516 - val_loss: 0.6773
Epoch 10/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1401 - val_lo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#finger 5

#model.fit with early stopping
model.fit(R2_train, y_train_2[:,4], epochs = 500, batch_size = 64, verbose = 1, validation_split = 0.1, callbacks = [early_stop])
model.save('model25.h5')
files.download('model25.h5')
#predictions
test_pred25 = model.predict(R2_test)

Epoch 1/500
85/85 [==============================] - 0s 3ms/step - loss: 0.4382 - val_loss: 0.7756
Epoch 2/500
85/85 [==============================] - 0s 3ms/step - loss: 0.3158 - val_loss: 0.7444
Epoch 3/500
85/85 [==============================] - 0s 3ms/step - loss: 0.2341 - val_loss: 0.7463
Epoch 4/500
85/85 [==============================] - 0s 3ms/step - loss: 0.1893 - val_loss: 0.7353
Epoch 5/500
85/85 [==============================] - 0s 3ms/step - loss: 0.1547 - val_loss: 0.7401
Epoch 6/500
85/85 [==============================] - 0s 3ms/step - loss: 0.1331 - val_loss: 0.7402
Epoch 7/500
85/85 [==============================] - 0s 3ms/step - loss: 0.1158 - val_loss: 0.7411
Epoch 8/500
85/85 [==============================] - 0s 3ms/step - loss: 0.1068 - val_loss: 0.7440
Epoch 9/500
85/85 [==============================] - 0s 3ms/step - loss: 0.0983 - val_loss: 0.7513
Epoch 10/500
85/85 [==============================] - 0s 3ms/step - loss: 0.0852 - val_loss: 0.7502
Epoch 11/

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#test_pred21 = test_pred21.reshape((-1,))
#test_pred22 = test_pred22.reshape((-1,))
#test_pred23 = test_pred23.reshape((-1,))
#test_pred24 = test_pred24.reshape((-1,))
#test_pred25 = test_pred25.reshape((-1,))

test_pred2 = np.vstack((test_pred21, test_pred22, test_pred23, test_pred24, test_pred25)).T

#interpolation
y2 = np.empty_like(test_pred2[:,0]) 
cs2 = [] 
for i in range(test_pred2.shape[1]):
  x2 = np.arange(test_pred2.shape[0])
  y2 = test_pred2[:,i]
  cs2.append(CubicSpline(x2, y2, bc_type = 'clamped'))

interp_pred2_nn = np.vstack((cs2[0](xs),cs2[1](xs), cs2[2](xs), cs2[3](xs), cs2[4](xs))).T

## subject 3

In [ ]:
#-----------Subject 3-------------

#scaling (normalizing) the data
scaler3 = StandardScaler()
R3_train = scaler3.fit_transform(R3_train)

scaler_filename = "scaler3.save"
joblib.dump(scaler3, scaler_filename) 

R3_test = scaler3.transform(R3_test) 

In [ ]:
#creating the neural network architecture
model = Sequential()
model.add(Dense(128, input_dim = R3_train.shape[1], activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(1, activation = 'linear'))

#choosing optimizer and loss
model.compile(optimizer = 'adam', loss = 'mean_absolute_error')

#early stopping
early_stop = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose=1, patience = 25)

In [ ]:
#finger 1

#model.fit with early stopping
model.fit(R3_train, y_train_3[:,0], epochs = 500, batch_size = 32, verbose = 1, validation_split = 0.1, callbacks = [early_stop])
model.save('model31.h5')
files.download('model31.h5')
#predictions
test_pred31 = model.predict(R3_test)

Epoch 1/500
169/169 [==============================] - 1s 3ms/step - loss: 0.6037 - val_loss: 0.3487
Epoch 2/500
169/169 [==============================] - 0s 3ms/step - loss: 0.4105 - val_loss: 0.3799
Epoch 3/500
169/169 [==============================] - 0s 3ms/step - loss: 0.3084 - val_loss: 0.2913
Epoch 4/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2470 - val_loss: 0.3208
Epoch 5/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2034 - val_loss: 0.3236
Epoch 6/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1827 - val_loss: 0.3127
Epoch 7/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1602 - val_loss: 0.3111
Epoch 8/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1488 - val_loss: 0.3251
Epoch 9/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1333 - val_loss: 0.3057
Epoch 10/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1272 - val_lo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#finger 2

#model.fit with early stopping
model.fit(R3_train, y_train_3[:,1], epochs = 500, batch_size = 32, verbose = 1, validation_split = 0.1, callbacks = [early_stop])
model.save('model32.h5')
files.download('model32.h5')
#predictions
test_pred32 = model.predict(R3_test)

Epoch 1/500
169/169 [==============================] - 1s 3ms/step - loss: 0.6983 - val_loss: 0.3657
Epoch 2/500
169/169 [==============================] - 0s 3ms/step - loss: 0.4478 - val_loss: 0.4175
Epoch 3/500
169/169 [==============================] - 0s 3ms/step - loss: 0.3353 - val_loss: 0.3964
Epoch 4/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2707 - val_loss: 0.4354
Epoch 5/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2278 - val_loss: 0.4444
Epoch 6/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1908 - val_loss: 0.4230
Epoch 7/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1723 - val_loss: 0.4337
Epoch 8/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1572 - val_loss: 0.4740
Epoch 9/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1416 - val_loss: 0.4584
Epoch 10/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1317 - val_lo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#finger 3

#model.fit with early stopping
model.fit(R3_train, y_train_3[:,2], epochs = 500, batch_size = 32, verbose = 1, validation_split = 0.1, callbacks = [early_stop])
model.save('model33.h5')
files.download('model33.h5')
#predictions
test_pred33 = model.predict(R3_test)

Epoch 1/500
169/169 [==============================] - 0s 3ms/step - loss: 0.4841 - val_loss: 0.6712
Epoch 2/500
169/169 [==============================] - 0s 3ms/step - loss: 0.3377 - val_loss: 0.6648
Epoch 3/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2560 - val_loss: 0.6585
Epoch 4/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2094 - val_loss: 0.6584
Epoch 5/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1785 - val_loss: 0.7090
Epoch 6/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1531 - val_loss: 0.6724
Epoch 7/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1333 - val_loss: 0.6967
Epoch 8/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1200 - val_loss: 0.6715
Epoch 9/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1127 - val_loss: 0.6606
Epoch 10/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1008 - val_lo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#finger 4

#model.fit with early stopping
model.fit(R3_train, y_train_3[:,3], epochs = 500, batch_size = 32, verbose = 1, validation_split = 0.1, callbacks = [early_stop])
model.save('model34.h5')
files.download('model34.h5')
#predictions
test_pred34 = model.predict(R3_test)

Epoch 1/500
169/169 [==============================] - 0s 3ms/step - loss: 0.5283 - val_loss: 0.5078
Epoch 2/500
169/169 [==============================] - 0s 3ms/step - loss: 0.3131 - val_loss: 0.5552
Epoch 3/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2329 - val_loss: 0.5327
Epoch 4/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1779 - val_loss: 0.5455
Epoch 5/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1482 - val_loss: 0.5137
Epoch 6/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1310 - val_loss: 0.5625
Epoch 7/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1147 - val_loss: 0.5750
Epoch 8/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1047 - val_loss: 0.5384
Epoch 9/500
169/169 [==============================] - 0s 3ms/step - loss: 0.0970 - val_loss: 0.5600
Epoch 10/500
169/169 [==============================] - 0s 3ms/step - loss: 0.0897 - val_lo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#finger 5

#model.fit with early stopping
model.fit(R3_train, y_train_3[:,4], epochs = 500, batch_size = 32, verbose = 1, validation_split = 0.1, callbacks = [early_stop])
model.save('model35.h5')
files.download('model35.h5')
#predictions
test_pred35 = model.predict(R3_test)

Epoch 1/500
169/169 [==============================] - 0s 3ms/step - loss: 0.3925 - val_loss: 0.4085
Epoch 2/500
169/169 [==============================] - 0s 3ms/step - loss: 0.2441 - val_loss: 0.4815
Epoch 3/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1720 - val_loss: 0.3741
Epoch 4/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1305 - val_loss: 0.4136
Epoch 5/500
169/169 [==============================] - 0s 3ms/step - loss: 0.1143 - val_loss: 0.4721
Epoch 6/500
169/169 [==============================] - 0s 3ms/step - loss: 0.0946 - val_loss: 0.3751
Epoch 7/500
169/169 [==============================] - 0s 3ms/step - loss: 0.0912 - val_loss: 0.4347
Epoch 8/500
169/169 [==============================] - 0s 3ms/step - loss: 0.0830 - val_loss: 0.4462
Epoch 9/500
169/169 [==============================] - 0s 3ms/step - loss: 0.0699 - val_loss: 0.4685
Epoch 10/500
169/169 [==============================] - 0s 3ms/step - loss: 0.0676 - val_lo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#test_pred31 = test_pred31.reshape((-1,))
#test_pred32 = test_pred32.reshape((-1,))
#test_pred33 = test_pred33.reshape((-1,))
#test_pred34 = test_pred34.reshape((-1,))
#test_pred35 = test_pred35.reshape((-1,))

test_pred3 = np.vstack((test_pred31, test_pred32, test_pred33, test_pred34, test_pred35)).T

#interpolation
y3 = np.empty_like(test_pred3[:,0]) 
cs3 = [] 
for i in range(test_pred3.shape[1]):
  x3 = np.arange(test_pred3.shape[0])
  y3 = test_pred3[:,i]
  cs3.append(CubicSpline(x3, y3, bc_type = 'clamped'))

interp_pred3_nn = np.vstack((cs3[0](xs),cs3[1](xs), cs3[2](xs), cs3[3](xs), cs3[4](xs))).T

# Stacking all predictions 

In [ ]:
#-------STACKING ALL PREDICTIONS-----------
predictions = np.zeros((3,1), dtype=object)
predictions[0,0] = interp_pred1_nn
predictions[1,0] = interp_pred2_nn
predictions[2,0] = interp_pred3_nn

savemat('predictions.mat', {'predicted_dg':predictions})

In [ ]:
files.download('predictions.mat')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>